# NOPE through IB

+ ensure that you have TWS and running, API enabled etc...
+ **also ensure that you have correct mkt data subscriptions** (find out in TWS by right click on your preffered contract (SPY for example) -> Launch Market Data subscription manager
    + for SPY you need:
        1. NYSE American, BATS, ARCA, IEX, and Regional Exchanges (Network B)
        1. OPRA (US Options Exchanges)

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()

In [2]:
if not ib.isConnected():
    print('connecting')
    ib.connect('127.0.0.1', 7497, clientId=1, readonly=True)
    if ib.isConnected():
        ib.reqMarketDataType(1)
        print('connected')
    else:
        print('Error, try again or smthing')
else:
    print('already connected')




connecting
connected


In [27]:
import pickle
from tqdm.notebook import tqdm
from bisect import bisect

def find_options(underlying: Contract):
    print('Getting market data')
    underlying_data = ib.reqTickers(underlying)
    if not underlying_data:
        raise ValueError('Couldnt get market data for your contract')
    else:
        print('Got market data')

    underlying_data = underlying_data[0]

    option_params = ib.reqSecDefOptParams(underlying.symbol, '', underlying.secType, underlying.conId)

    qualifiedContracts = {'CALL': [], 'PUT': []}

    # you might want to discard weeklies (SPXW)
    print('Ignore errors, theres no way to safely check if combination of expiry and strike exists\nJust wait till tapes bar go to 100%')
    for i, opt_chain in enumerate(tqdm(option_params, desc='Tapes: ')):
        opt_chain.strikes.sort()
        strike_close_to_last = bisect(opt_chain.strikes, underlying_data.last)
        if strike_close_to_last == 0:
            strike_close_to_last += 1 # unsafely assume that there are more than two strikes so we can split the array in two
        for j, expiration in enumerate(tqdm(opt_chain.expirations, desc='Expirations: ')):
            for right in ('CALL', 'PUT'):
                # TODO figure out how to merge these two loops
                for strike_idx in range(strike_close_to_last, len(opt_chain.strikes)):
                    strike = opt_chain.strikes[strike_idx]
                    print(f"\rGetting strike: {strike}", end="")
                    contracts = ib.qualifyContracts(Option(underlying.symbol, expiration, strike, right, opt_chain.exchange))
                    if contracts:
                        qualifiedContracts[right].append(contracts[0])
                    else:
                        break
                for strike_idx in range(strike_close_to_last - 1, -1, -1):
                    strike = opt_chain.strikes[strike_idx]
                    print(f"\rGetting strike: {strike}", end="")
                    contracts = ib.qualifyContracts(Option(underlying.symbol, expiration, strike, right, opt_chain.exchange))
                    if contracts:
                        qualifiedContracts[right].append(contracts[0])
                    else:
                        break
    return qualifiedContracts

def save_contracts(a):
    with open('cached_contracts.pkl', 'wb') as handle:
        pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_contracts():
    with open('cached_contracts.pkl', 'rb') as handle:
        return pickle.load(handle)

Which contract do you want to calculate NOPE on?

In [ ]:
# you can also use ASX exchange or track SPX using Index('SPX', 'CBOE')
underlying = Stock('SPY', 'ARCA')
underlying = ib.qualifyContracts(underlying)
if underlying:
    underlying = underlying[0]
else:
    raise ValueError('Underlying contract not, found see errors from logger. Please supply valid contract')

## PAY ATTENTION HERE
It takes awfully long time to get all possible SPY options. I highly recommend you to cache them and load them from pickle

Someone with more financial background might do better assumptions on what is needless to check. Until such hero arrives we are stuck with brute force.

In [ ]:
# UNCOMMENT TWO FOLLOWING LINES IF YOU WANT TO GET/REFRESH contracts
# qualifiedContracts = find_options(underlying)
# save_contracts(qualifiedContracts)

# UNCOMMENT FOLLOWING LINE IF YOU WANT TO LOAD THEM FROM CACHE
# qualifiedContracts = load_contracts()

In [30]:
# FOLLOWING CODE ASSUMES EXISTENCE OF REGISTER
# this is bad but in following code we will assume that before onPendingTickers gets called REGISTER will be seeded with correct values
REGISTER = {}

def register_option_volumes(options):
    print('Registering subscriptions')
    for right in ('CALL', 'PUT'):
        for ticker in tqdm(options[right], desc=f'Registering {right} listeners: '):
            ib.reqMktData(ticker, '100', False, False)
            REGISTER['option_vols'][ticker.conId] = 0
    print('Registred')

def onPendingTickers(tickers):
    for ticker in tickers:
        if ticker.contract.conId == REGISTER['underlying_conId']:
            REGISTER['underlying_vol'] = ticker.volume # todo check if not nan perhaps
        if ticker.contract.conId in REGISTER['option_vols']:
            direction = 1
            if ticker.contract.right == 'PUT':
                direction = -1
            new_volume = direction * ticker.volume * ticker.modelGreeks.delta
            # new volume = remove old volume and add new volume
            REGISTER['opt_total_volume'] += direction * -1 * REGISTER['option_vols'][ticker.contract.conId] + new_volume
            REGISTER['option_vols'][ticker.contract.conId] = new_volume
            print(f'NOPE IS: {REGISTER["opt_total_volume"] / REGISTER["underlying_vol"]}, sanity: {REGISTER["opt_total_volume"]} / {REGISTER["underlying_vol"]}')

In [32]:
# MAIN LOOP AHEAD

print(f'Qualified {len(qualifiedContracts["CALL"]) + len(qualifiedContracts["PUT"])} contracts')
print(f'Underlying ticker is {underlying.symbol}, track with con id: {underlying.conId}')

# subscribe to underlyings market data
ib.reqMktData(underlying, '100')
REGISTER['underlying_conId'] = underlying.conId
REGISTER['underlying_vol'] = 0 # aka denominator of NOPE equation

# here we will store last option volumes
REGISTER['option_vols'] = {}
option_conn_ids = register_option_volumes(qualifiedContracts)

# this is the nominator of the NOPE equation
REGISTER['opt_total_volume'] = 0

ib.pendingTickersEvent += onPendingTickers

Qualified 15096 contracts
Underlying ticker is SPY, track with con id: 756733
Registering subscriptions
Registred


Registering CALL listeners:   0%|          | 0/7548 [00:00<?, ?it/s]

Registering PUT listeners:   0%|          | 0/7548 [00:00<?, ?it/s]

Error 101, reqId 25508: Max number of tickers has been reached, contract: Option(conId=467163013, symbol='SPY', lastTradeDateOrContractMonth='20210305', strike=375.0, right='C', multiplier='100', exchange='NASDAQOM', currency='USD', localSymbol='SPY   210305C00375000', tradingClass='SPY')
Error 101, reqId 25475: Max number of tickers has been reached, contract: Option(conId=469567988, symbol='SPY', lastTradeDateOrContractMonth='20210305', strike=401.0, right='C', multiplier='100', exchange='NASDAQOM', currency='USD', localSymbol='SPY   210305C00401000', tradingClass='SPY')
Error 101, reqId 25464: Max number of tickers has been reached, contract: Option(conId=467163258, symbol='SPY', lastTradeDateOrContractMonth='20210305', strike=392.0, right='C', multiplier='100', exchange='NASDAQOM', currency='USD', localSymbol='SPY   210305C00392000', tradingClass='SPY')
Error 101, reqId 25492: Max number of tickers has been reached, contract: Option(conId=467163209, symbol='SPY', lastTradeDateOrCon

In [34]:
ib.disconnect()